# Chapter 6: Working with Sets

The basic set operations are `UNION [ALL]` and `INTERSECT`.

Set precedence is important, but the ANSI 92 standard dictates that `INTERSECT` takes highest precedence. Multiple queries are combined using parentheses.

# Exercises

## 6-2
Write a compound query that finds the first and last names of individual customers along with the first and last names of all employees

## 6-3
Sort the results from exercise 6-2 by the name column.


In [1]:
import os

from dotenv import load_dotenv
from sqlalchemy import create_engine, URL, select, func
from sqlalchemy.orm import Session
import pandas as pd

from utils import print_sql_statement


load_dotenv()

url_object = URL.create(
    os.environ["DB_ENGINE"],
    username=os.environ["DB_USER"],
    password=os.environ["DB_PASSWD"],
    host=os.environ["DB_HOST"],
    database=os.environ["DB_NAME"],
)

engine = create_engine(url_object)

In [23]:
from model import Employee, Individual
# Combine 6-2 and 6-3


with Session(engine) as session:

    df = pd.read_sql_query(
        """
        SELECT
            i.fname
            , i.lname
        FROM
            individual i
        UNION ALL
        (
            SELECT
                fname
                , lname
            FROM
                employee
        )
        ORDER BY lname;
        """,
        con=session.connection()
    )

    statement = (
        select(
            Individual.fname,
            Individual.lname
        )
        .union(
            select(
                Employee.fname,
                Employee.lname
            )
        )
        .order_by(Individual.lname)
    )
    print_sql_statement(statement)
    results = session.execute(statement).all()

    new_results = sorted(
        [
            (opt.fname, opt.lname)
            for entity in (Individual, Employee)
            for opt in session.query(entity)
        ],
        key=lambda tup: tup[-1]
    )

print(df)
assert results == new_results

"""SELECT individual.fname, individual.lname 
FROM individual UNION SELECT employee.fname, employee.lname 
FROM employee ORDER BY lname"""
       fname      lname
0      Susan     Barker
1      Louis      Blake
2       John      Blake
3    Richard     Farley
4      Helen    Fleming
5       Beth     Fowler
6    Charles    Frasier
7       John    Gooding
8       Jane   Grossman
9      James     Hadley
10     Susan  Hawthorne
11      John    Hayward
12  Samantha    Jameson
13   Theresa    Markham
14     Cindy      Mason
15     Sarah     Parker
16     Frank    Portman
17     Paula    Roberts
18   Michael      Smith
19      John    Spencer
20     Susan    Tingley
21     Frank     Tucker
22     Chris     Tucker
23      Rick     Tulman
24    Robert      Tyler
25  Margaret      Young
26    Thomas    Ziegler
